## Building Big Data Anomaly Detection With Hadoop

### Detecting Network Intrusions

#### Data Ingestion

We have downloaded KDD-99 dataset from 'http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html', We need to import this data to hive table to create a model for Anomaly Detction System.

In 1999, the original TCP dump files were pre-processed for utilisation in the KDD intrusion detection benchmark and competetion, reulting is what known as KDD-99 dataset.

#### Feature Description

duration:continuous.
protocol_type: symbolic.
service: symbolic.
flag: symbolic.
src_bytes: continuous.
dst_bytes: continuous.
land: symbolic.
wrong_fragment: continuous.
urgent: continuous.
hot: continuous.
num_failed_logins: continuous.
logged_in: symbolic.
num_compromised: continuous.
root_shell: continuous.
su_attempted: continuous.
num_root: continuous.
num_file_creations: continuous.
num_shells: continuous.
num_access_files: continuous.
num_outbound_cmds: continuous.
is_host_login: symbolic.
is_guest_login: symbolic.
count: continuous.
srv_count: continuous.
serror_rate: continuous.
srv_serror_rate: continuous.
rerror_rate: continuous.
srv_rerror_rate: continuous.
same_srv_rate: continuous.
diff_srv_rate: continuous.
srv_diff_host_rate: continuous.
dst_host_count: continuous.
dst_host_srv_count: continuous.
dst_host_same_srv_rate: continuous.
dst_host_diff_srv_rate: continuous.
dst_host_same_src_port_rate: continuous.
dst_host_srv_diff_host_rate: continuous.
dst_host_serror_rate: continuous.
dst_host_srv_serror_rate: continuous.
dst_host_rerror_rate: continuous.
dst_host_srv_rerror_rate: continuous.

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.config("spark.sql.warehouse.dir","/user/itv000684/warehouse").enableHiveSupport().appName("Intrusion Detection System").master("yarn").getOrCreate()

In [3]:
spark.sql("""DROP DATABASE IF EXISTS itv000684_kdd99data CASCADE""")

""


In [4]:
spark.sql("CREATE DATABASE IF NOT EXISTS itv000684_kdd99data")

""


In [5]:
spark.sql("""USE itv000684_kdd99data""")

""


In [6]:
spark.sql("SELECT current_database()")

current_database()
itv000684_kdd99data


In [7]:
spark.sql("DROP TABLE IF EXISTS kdd99raw")

""


In [8]:
spark.sql("""CREATE EXTERNAL TABLE kdd99raw (
duration  INT,
protocol_type  STRING,
service  STRING,
flag  STRING,
src_bytes  INT,
dst_bytes  INT,
land  STRING,
wrong_fragment  INT,
urgent  INT,
hot  INT,
num_failed_logins  INT,
logged_in  STRING,
num_compromised  INT,
root_shell  INT,
su_attempted  INT,
num_root  INT,
num_file_creations  INT,
num_shells  INT,
num_access_files  INT,
num_outbound_cmds  INT,
is_host_login  STRING,
is_guest_login  STRING,
count  INT,
srv_count  INT,
serror_rate  FLOAT,
srv_serror_rate  FLOAT,
rerror_rate  FLOAT,
srv_rerror_rate  FLOAT,
same_srv_rate  FLOAT,
diff_srv_rate  FLOAT,
srv_diff_host_rate  FLOAT,
dst_host_count  FLOAT,
dst_host_srv_count  FLOAT,
dst_host_same_srv_rate  FLOAT,
dst_host_diff_srv_rate  FLOAT,
dst_host_same_src_port_rate  FLOAT,
dst_host_srv_diff_host_rate  FLOAT,
dst_host_serror_rate  FLOAT,
dst_host_srv_serror_rate  FLOAT,
dst_host_rerror_rate  FLOAT,
dst_host_srv_rerror_rate  FLOAT,
is_anomaly STRING) ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
STORED AS TEXTFILE
LOCATION '/user/itv000684/kdd'
""")


""


In [9]:
spark.sql("DESCRIBE FORMATTED kdd99raw").show(50)

+--------------------+--------------------+-----------------+
|            col_name|           data_type|          comment|
+--------------------+--------------------+-----------------+
|            duration|              string|from deserializer|
|       protocol_type|              string|from deserializer|
|             service|              string|from deserializer|
|                flag|              string|from deserializer|
|           src_bytes|              string|from deserializer|
|           dst_bytes|              string|from deserializer|
|                land|              string|from deserializer|
|      wrong_fragment|              string|from deserializer|
|              urgent|              string|from deserializer|
|                 hot|              string|from deserializer|
|   num_failed_logins|              string|from deserializer|
|           logged_in|              string|from deserializer|
|     num_compromised|              string|from deserializer|
|       

In [10]:
spark.sql("""CREATE TABLE kdd99 STORED AS ORC tblproperties ("orc.compress" = "ZLIB")
AS SELECT
        protocol_type, service, flag, is_anomaly,
        CAST(duration AS  INT),
        CAST(src_bytes AS  INT),
        CAST(dst_bytes AS  INT),
        CAST(wrong_fragment AS  INT),
        CAST(urgent AS  INT),
        CAST(hot AS  INT),
        CAST(num_failed_logins AS  INT),
        CAST(num_compromised AS  INT),
        CAST(root_shell AS  INT),
        CAST(su_attempted AS  INT),
        CAST(num_root AS  INT),
        CAST(num_file_creations AS  INT),
        CAST(num_shells AS  INT),
        CAST(num_access_files AS  INT),
        CAST(num_outbound_cmds AS  INT),
        CAST(count AS  INT),
        CAST(srv_count AS  INT),
        CAST(serror_rate AS  FLOAT),
        CAST(srv_serror_rate AS  FLOAT),
        CAST(rerror_rate AS  FLOAT),
        CAST(srv_rerror_rate AS  FLOAT),
        CAST(same_srv_rate AS  FLOAT),
        CAST(diff_srv_rate AS  FLOAT),
        CAST(srv_diff_host_rate AS  FLOAT),
        CAST(dst_host_count AS  FLOAT),
        CAST(dst_host_srv_count AS  FLOAT),
        CAST(dst_host_same_srv_rate AS  FLOAT),
        CAST(dst_host_diff_srv_rate AS  FLOAT),
        CAST(dst_host_same_src_port_rate AS  FLOAT),
        CAST(dst_host_srv_diff_host_rate AS  FLOAT),
        CAST(dst_host_serror_rate AS  FLOAT),
        CAST(dst_host_srv_serror_rate AS  FLOAT),
        CAST(dst_host_rerror_rate AS  FLOAT),
        CAST(dst_host_srv_rerror_rate AS  FLOAT)
FROM kdd99raw
""")

""


In [11]:
spark.sql("SELECT * FROM kdd99").show()

+-------------+-------+----+----------+--------+---------+---------+--------------+------+---+-----------------+---------------+----------+------------+--------+------------------+----------+----------------+-----------------+-----+---------+-----------+---------------+-----------+---------------+-------------+-------------+------------------+--------------+------------------+----------------------+----------------------+---------------------------+---------------------------+--------------------+------------------------+--------------------+------------------------+
|protocol_type|service|flag|is_anomaly|duration|src_bytes|dst_bytes|wrong_fragment|urgent|hot|num_failed_logins|num_compromised|root_shell|su_attempted|num_root|num_file_creations|num_shells|num_access_files|num_outbound_cmds|count|srv_count|serror_rate|srv_serror_rate|rerror_rate|srv_rerror_rate|same_srv_rate|diff_srv_rate|srv_diff_host_rate|dst_host_count|dst_host_srv_count|dst_host_same_srv_rate|dst_host_diff_srv_rate|d